# Notebook which mimics Ovation Prime output available from [ISWA Widget](http://iswa.gsfc.nasa.gov/IswaSystemWebApp/index.jsp?i_1=457&l_1=60&t_1=290&w_1=500&h_1=510&s_1=0_0_10_3&i_2=451&l_2=573&t_2=293&w_2=500&h_2=510&s_2=0_0_10_3&i_3=465&l_3=55&t_3=810&w_3=500&h_3=510&s_3=0_0_10_3&i_4=453&l_4=564&t_4=804&w_4=500&h_4=510&s_4=0_0_10_3&i_5=463&l_5=58&t_5=1327&w_5=500&h_5=510&s_5=0_0_10_3&i_6=459&l_6=568&t_6=1320&w_6=500&h_6=510&s_6=0_0_10_3&i_7=467&l_7=59&t_7=1850&w_7=500&h_7=510&s_7=0_0_10_3&i_8=461&l_8=570&t_8=1843&w_8=500&h_8=510&s_8=0_0_10_3&i_9=455&l_9=1077&t_9=294&w_9=500&h_9=510&s_9=0_0_10_3&i_10=510&l_10=1626&t_10=293&w_10=500&h_10=510&s_10=0_0_10_3&i_11=516&l_11=1073&t_11=811&w_11=500&h_11=510&s_11=0_0_10_3&i_12=503&l_12=1614&t_12=816&w_12=500&h_12=510&s_12=0_0_10_3&i_13=512&l_13=1076&t_13=1329&w_13=500&h_13=510&s_13=0_0_10_3&i_14=517&l_14=1602&t_14=1337&w_14=500&h_14=510&s_14=0_0_10_3&i_15=506&l_15=1080&t_15=1855&w_15=500&h_15=510&s_15=0_0_10_3&i_16=518&l_16=1595&t_16=1866&w_16=500&h_16=510&s_16=0_0_10_3&i_17=513&l_17=56&t_17=2380&w_17=500&h_17=510&s_17=0_0_10_3&i_18=507&l_18=574&t_18=2378&w_18=500&h_18=510&s_18=0_0_10_3&i_19=508&l_19=1073&t_19=2379&w_19=500&h_19=510&s_19=0_0_10_3&i_20=519&l_20=1587&t_20=2378&w_20=500&h_20=510&s_20=0_0_10_3&i_21=514&l_21=55&t_21=2897&w_21=500&h_21=510&s_21=0_0_10_3&i_22=515&l_22=562&t_22=2899&w_22=500&h_22=510&s_22=0_0_10_3&i_23=509&l_23=1066&t_23=2901&w_23=500&h_23=510&s_23=0_0_10_3&i_24=520&l_24=1566&t_24=2898&w_24=500&h_24=510&s_24=0_0_10_3)

The ISWA Ovation Prime Widget hosted at CCMC provides total electron energy flux (the sum of diffuse, monoenergy and wave/broadband), and hemispheric power (the area integral of the total electron energy flux). This notebook emulates this type of output for comparison with the ISWA widget (which implements the IDL code on which OvationPyme was originally based). 

    NOTE: This notebook requires geospacepy (which is not a dependancy of OvationPyme) to be installed (to calculate hemispheric power by integrating energy flux). You can pip install geospacepy to get this notebook working.

In [ ]:
import numpy as np
import datetime
import matplotlib.pyplot as plt
from ovationpyme.ovation_prime import FluxEstimator,AverageEnergyEstimator,BinCorrector
from ovationpyme.ovation_utilities import calc_avg_solarwind
from ovationpyme.ovation_plotting import latlt2polar,polar2dial,pcolor_flux
from geospacepy.spherical_geometry import grid_surface_integral
import ipywidgets
from collections import OrderedDict
from functools import partial

auroral_types = ['diff','mono','wave','ions']
flux_type = "energy"

def plot_date_fluxes_callback(clicked_button,widgets):
    f = plt.figure(figsize=(12,6),dpi=150)
    axN = f.add_subplot(121,projection='polar')
    axS = f.add_subplot(122,projection='polar')
    atypes = widgets['atype_select'].value
    bincorrect = widgets['bincorrect_toggle'].value
    combine_hemispheres = widgets['combineNS_toggle'].value
    manual_newell = widgets['newell_float'].value
    year = widgets['year'].value
    month = widgets['month'].value
    
    day = widgets['day'].value
    hour = widgets['hour'].value
    minute = widgets['minute'].value
    dt = datetime.datetime(year,month,day,hour,minute)
    dtstr = dt.strftime('%Y%m%d %H:%M')
    estimators = {atype:FluxEstimator(atype,flux_type) for atype in atypes}
    if manual_newell != -1.:
        estimator._dF = manual_newell
    tflux_kwargs = {'combine_hemispheres':combine_hemispheres,
                    'return_dF':True}
    
    fluxgridsN,fluxgridsS = [],[]
    for i,(atype,estimator) in enumerate(estimators.items()):
        fluxtupleN = estimator.get_flux_for_time(dt,hemi='N',**tflux_kwargs)
        fluxtupleS = estimator.get_flux_for_time(dt,hemi='S',**tflux_kwargs)
        
        if i==1:
            mlatgridN,mltgridN = fluxtupleN[0],fluxtupleN[1]
            mlatgridS,mltgridS = fluxtupleS[0],fluxtupleS[1]
            newell_coupling = fluxtupleN[3]
            
        fluxgridsN.append(fluxtupleN[2])
        fluxgridsS.append(fluxtupleS[2])
        
    fluxgridN = np.sqrt(np.add.reduce([Nflux**2 for Nflux in fluxgridsN]))
    fluxgridS = np.sqrt(np.add.reduce([Sflux**2 for Sflux in fluxgridsS]))
    
    HPN = grid_surface_integral(mlatgridN,mltgridN,fluxgridN,6371200,'hour')/1000/1.0e9
    HPS = grid_surface_integral(mlatgridS,mltgridS,fluxgridS,6371200,'hour')/1000/1.0e9
    
    if bincorrect:
        bcN = BinCorrector(mlatgridN,mltgridN)
        fluxgridN = bcN.fix(fluxgridN)
        bcS = BinCorrector(mlatgridS,mltgridS)
        fluxgridS = bcS.fix(fluxgridS)
        print("Correction Applied")
    pcolor_kwargs = {'cmap':'gnuplot'}
    manual_vmin = wgts['vmin_float'].value
    if manual_vmin != -1.:
        pcolor_kwargs['vmin']=manual_vmin
    manual_vmax = wgts['vmax_float'].value
    if manual_vmax != -1.:
        pcolor_kwargs['vmax']=manual_vmax
    mappableN = pcolor_flux(axN,mlatgridN,mltgridN,fluxgridN,'N',**pcolor_kwargs)
    mappableS = pcolor_flux(axS,mlatgridS,mltgridS,fluxgridS,'S',**pcolor_kwargs)
    f.colorbar(mappableN,ax=axN,label=flux_type)
    f.colorbar(mappableS,ax=axS,label=flux_type)
    axN.set_title('Northern Hemisphere {:.1f} GW'.format(HPN))
    axS.set_title('Southern Hemisphere {:.1f} GW'.format(HPS))
    polar2dial(axN)
    polar2dial(axS)
    f.suptitle(('%s, %s' % (atypes,flux_type)
                +'\nNewell Coupling (%s): %f' % (dtstr,newell_coupling)))
    plt.show()
    
wgts = OrderedDict()
wgts['year']=ipywidgets.IntText(description='Year',value=2010)
wgts['month']=ipywidgets.IntText(description='Month',value=5)
wgts['day']=ipywidgets.IntText(description='Day',value=29)
wgts['hour']=ipywidgets.IntText(description='Hour',value=12)
wgts['minute']=ipywidgets.IntText(description='Minute',value=0)
wgts['atype_select']=ipywidgets.SelectMultiple(value=['diff','mono','wave'],options=auroral_types,description='[Ctrl+Click] Selects Multiple')
wgts['bincorrect_toggle']=ipywidgets.ToggleButton(description='Correction',value=False)
wgts['combineNS_toggle']=ipywidgets.ToggleButton(description='Combine N&S',value=True)
wgts['newell_float']=ipywidgets.FloatText(description='Newell Coupling (-1 to use date)',value=-1)
wgts['vmin_float']=ipywidgets.FloatText(description='Colorbar Min (-1 is Autoscale)',value=-1)
wgts['vmax_float']=ipywidgets.FloatText(description='Colorbar Max (-1 is Autoscale)',value=-1)
wgts['run_button'] = ipywidgets.Button(description='Calculate')
wgts['run_button'].on_click(partial(plot_date_fluxes_callback,widgets=wgts))

for widget_name,widget in wgts.items():
    display(widget)